In [41]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
from nltk.corpus import stopwords


In [35]:
def _file(file):
    file = open(file, "r", encoding="utf-8")
    text = file.read()
    file.close()
    return text

def clean_text(text, stops):
	res = ''.join(re.findall(r'[a-zA-z]| ', text))
	tokens = [i.lower() for i in res.split() if i not in stops]
	return ' '.join(tokens)

In [55]:
stops = stopwords.words()
df = pd.read_csv("train.csv")
df['text'] = df['text'].apply(clean_text, args=(stops,))
df

,id,text,answer
0,0,what disappointment admittedly best prequels s...,0
1,1,this pale imitation die hard franchise sucks t...,0
2,2,this goodguyvstheeviltyrant story set th centu...,0
3,3,this documentary i came across chance uk tv ch...,1
4,4,this installment masters horror terrible appar...,0
...,...,...,...
24995,24995,horrible script apparently directed byno the m...,0
24996,24996,five years tenko survivors returning home mari...,1
24997,24997,i dont understand not critic evaluating qualit...,1
24998,24998,this movie pretentious foppish right funny the...,0


In [121]:
review = df['text'].values
label = df['answer'].values
r_train, r_test, l_train, l_test = train_test_split(
    review, label, test_size=0.01, random_state=1000)

In [124]:
tokenizer = Tokenizer(num_words=25000)
tokenizer.fit_on_texts(r_train)
data_train = tokenizer.texts_to_sequences(r_train)
data_test = tokenizer.texts_to_sequences(r_test)
v_size = len(tokenizer.word_index) + 1  

In [125]:
Max = 250
data_train = pad_sequences(data_train, padding='post', maxlen=Max)
data_test = pad_sequences(data_test, padding='post', maxlen=Max)
embedding_dim = 200

In [126]:
model = Sequential([
    layers.Embedding(v_size, embedding_dim, input_length=Max),
    layers.Conv1D(128, 5, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(10, activation='relu'),
    layers.Dense(1, activation='sigmoid'),])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 250, 200)          24008000  
                                                                 
 conv1d_18 (Conv1D)          (None, 246, 128)          128128    
                                                                 
 global_max_pooling1d_18 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_36 (Dense)            (None, 10)                1290      
                                                                 
 dense_37 (Dense)            (None, 1)                 11        
                                                                 
Total params: 24,137,429
Trainable params: 24,137,429
Non-trainable params: 0
_________________________________________

In [127]:
model.fit(data_train, l_train, epochs=5, verbose=2, batch_size=30)
loss, accuracy = model.evaluate(data_test, l_test, verbose=2)
print(f"Test accuracy: {accuracy}")

Epoch 1/5
825/825 - 481s - loss: 0.3713 - accuracy: 0.8317 - 481s/epoch - 583ms/step
Epoch 2/5
825/825 - 511s - loss: 0.1139 - accuracy: 0.9616 - 511s/epoch - 620ms/step
Epoch 3/5
825/825 - 489s - loss: 0.0132 - accuracy: 0.9975 - 489s/epoch - 592ms/step
Epoch 4/5
825/825 - 466s - loss: 0.0013 - accuracy: 1.0000 - 466s/epoch - 565ms/step
Epoch 5/5
825/825 - 501s - loss: 2.5333e-04 - accuracy: 1.0000 - 501s/epoch - 607ms/step
8/8 - 1s - loss: 0.4025 - accuracy: 0.8760 - 646ms/epoch - 81ms/step
Test accuracy: 0.8759999871253967


In [128]:
df_test = pd.read_csv('test.csv')
test_data = df_test['text'].values
test_data = tokenizer.texts_to_sequences(test_data)
test_data = pad_sequences(test_data, padding='post', maxlen=Max)
predictions = model.predict(test_data)
predictions = predictions.tolist()
predictions = [round(i[0]) for i in predictions]
df_test['answer'] = predictions
del df_test['text']
df_test.to_csv('result_CNN.csv', index=False)